# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output data/cities.csv")

# Display sample data
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-3.41,83,98,6.75,SJ,1713735174
1,1,labrador city,52.9463,-66.9114,-0.05,43,40,3.09,CA,1713735010
2,2,port elizabeth,-33.9180,25.5701,20.61,96,20,4.12,ZA,1713735175
3,3,san antonio de pale,-1.4014,5.6325,28.99,81,70,2.85,GQ,1713735176
4,4,vilyuchinsk,52.9306,158.4028,2.15,85,66,0.37,RU,1713735176


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = 'True',
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 500,
    xlabel = "x",
    ylabel = "y",
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

C:\Users\colea\.conda\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria: A max temperature higher than 28, Zero cloudiness, Humidity less than or equal to 30%

revised_city_data_df = city_data_df[city_data_df['Max Temp']>28]
revised_city_data_df = revised_city_data_df[city_data_df['Cloudiness'] == 0]
revised_city_data_df = revised_city_data_df[city_data_df['Humidity']<=30]

# Display sample data
revised_city_data_df

C:\Users\colea\AppData\Local\Temp\ipykernel_17952\2125828604.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  revised_city_data_df = revised_city_data_df[city_data_df['Cloudiness'] == 0]
C:\Users\colea\AppData\Local\Temp\ipykernel_17952\2125828604.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  revised_city_data_df = revised_city_data_df[city_data_df['Humidity']<=30]


,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,cabo san lucas,22.8909,-109.9124,32.61,30,0,4.63,MX,1713734980
220,220,coyuca de catalan,18.3333,-100.6500,43.39,9,0,1.27,MX,1713735401
407,407,blythe,33.6103,-114.5964,36.25,11,0,3.09,US,1713735596
417,417,el bauga,18.2620,33.9081,28.63,7,0,7.21,SD,1713735607


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({'City': revised_city_data_df['City'],'Country':revised_city_data_df['Country'], 'Latitude': revised_city_data_df['Lat'],'Longitude': revised_city_data_df['Lng'],'Humidity': revised_city_data_df['Humidity']})
hotel_df

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity
89,cabo san lucas,MX,22.8909,-109.9124,30
220,coyuca de catalan,MX,18.3333,-100.6500,9
407,blythe,US,33.6103,-114.5964,11
417,el bauga,SD,18.2620,33.9081,7


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
#Get parameters to search for a hotel
radius = 10000

params = {'categories': 'accommodation',
            'api_key': f'{geoapify_key}'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Latitude']
    lng = row['Longitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
coyuca de catalan - nearest hotel: Hotel EL ENCANTO
blythe - nearest hotel: Comfort Suites
el bauga - nearest hotel: No hotel found


,City,Country,Latitude,Longitude,Humidity,Hotel Name
89,cabo san lucas,MX,22.8909,-109.9124,30,Comfort Rooms
220,coyuca de catalan,MX,18.3333,-100.6500,9,Hotel EL ENCANTO
407,blythe,US,33.6103,-114.5964,11,Comfort Suites
417,el bauga,SD,18.2620,33.9081,7,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:
# Configure the map plot

country = []
country = ['MX','MX','US','SD']
map_plot_1 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = 'True',
    tiles = 'EsriImagery',
    frame_width = 800,
    frame_height = 500,
    xlabel = "x",
    ylabel = "y",
    size = 500,
    color = 'City',
    line_color = 'black',   
    legend = 'bottom_right',
    hover_cols = ['Country','Hotel Name']
    
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Country,Hotel Name)